In [31]:
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [29]:
device = torch.device("cuda")

BATCH_SIZE = 64

In [30]:
def prepare_data(data_path='EEG_Data'):
    X = []
    Y = []

    for dir in os.listdir(data_path):
        folder_path = os.path.join(data_path, dir)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            spectrogram = np.load(file_path)
            X.append(spectrogram)
            Y.append(int(dir))
    X = np.array(X)
    Y = np.array(Y)
    
    X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [33]:
class EEG_Dataset(Dataset):
    def __init__(self, X, Y):
        
        self.X = X
        self.Y = Y
        self.X = torch.FloatTensor(X)
        self.Y = torch.LongTensor(Y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].squeeze().to(device), self.Y[idx].to(device)

In [34]:
X_train, X_val, X_test, y_train, y_val, y_test = prepare_data()

In [35]:
train_dataset = EEG_Dataset(X_train, y_train)
val_dataset = EEG_Dataset(X_val, y_val)
test_dataset = EEG_Dataset(X_test, y_test)

In [36]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)